In [1]:
#Importing of necessary packages
import nltk
import re
import emoji
import demoji
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from datetime import datetime

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertModel, pipeline
from numpy import loadtxt, savetxt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, BatchNormalization, Bidirectional
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score

#import bayes opt for hyperparameter tuning
from bayes_opt import BayesianOptimization

In [2]:
# Reading in the dataset
df_tweets = pd.read_csv('tweets_dataset_processed.csv')

In [3]:
df_tweets = df_tweets[["cleaned_text", "isBot"]]
df_tweets

,cleaned_text,isBot
0,aleah is me,0
1,__user_mention__ I got you bruh,0
2,__user_mention__ its a diet where u can only e...,1
3,When we are no longer able to change a situati...,1
4,__user_mention__ __user_mention__ __user_menti...,0
...,...,...
49995,__user_mention__ jfc whats wrong with the foru...,0
49996,__user_mention__ die hard with a vengeance __u...,0
49997,What would you do.... IF! a little old man pic...,1
49998,__user_mention__ lucky you..I can't unless if ...,1


In [4]:
n_samples = int(0.25 * len(df_tweets))
df_tweets_searches = resample(df_tweets, replace = False, n_samples = n_samples, random_state = 101)
df_tweets_searches = df_tweets_searches.reset_index(inplace = False, drop = True)
df_tweets_searches

,cleaned_text,isBot
0,__user_mention__ I'm up at 6am every day.. It sux,0
1,"Attitude stinks, but what point do i have to p...",1
2,"__user_mention__ god, I've got to agree with a...",1
3,__user_mention__ I understand though. For a fe...,0
4,Damn Laura STILL in there eatin,0
...,...,...
12495,__user_mention__ you already know... I'm VIP,0
12496,The Week in Pictures: June 21 - 28 via __user...,1
12497,Education Advisors Will Help You: Find a degre...,1
12498,RT __user_mention__ “@blackswan305: Who WOULDN...,1


# Model
1. Logistic Regression (BoW)
2. Random Forest Classifier (BoW)
3. Logistic Regression (TF-IDF)
4. Random Forest Classifier (TF-IDF)
5. LSTM
6. Logistic Regression (BERT)
7. Random Forest Classifier (BERT)
8. Adaboost Classifier (BERT)
9. XGBoost (BERT)

## BERT Models (Tuning)

In [5]:
#initialising a pretrained bert model
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", padding = True)
model = BertModel.from_pretrained("bert-base-cased")
nlp = pipeline("feature-extraction", tokenizer = tokenizer, model = model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
#store all the tweets after they have been encoded by BERT

# feature_list = []

# for index, row in df_tweets_searches.iterrows():
#     #extracting the ith tweet and restricting the characters to 512, which is fine because twitter's limit is 280
#     text = row['cleaned_text'][:512]
#     #encoding all the individual words present in the tweet
#     vec = np.array(nlp(text))
#     #getting the mean representation of the words present in the tweet
#     vec = vec.reshape((vec.shape[1], vec.shape[2])).mean(axis = 0)
#     feature_list.append(vec)

# feature_vectors_searches = np.array(feature_list)

In [7]:
feature_vectors_searches = pd.read_csv("feature_vect_searches.csv")
feature_vectors_searches = feature_vectors_searches.drop(columns = "Unnamed: 0")

In [8]:
feature_vectors_searches

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.277880,0.262478,0.010301,0.044697,0.332548,-0.044443,0.153921,-0.158961,-0.002900,-0.049170,...,-0.090998,0.140967,-0.202905,-0.195144,-0.004835,-0.132333,-0.120175,0.038537,-0.031248,0.512940
1,0.205477,0.086723,0.100212,-0.188577,0.081971,0.258901,0.285367,0.140156,-0.053721,-0.054970,...,-0.222869,-0.253721,-0.164111,0.022392,-0.381319,-0.180698,-0.026798,0.693803,0.317107,0.157766
2,0.299577,0.499393,0.003503,0.159204,0.239811,-0.370334,0.107382,-0.068318,0.180709,-0.256709,...,-0.065183,0.190202,-0.246934,-0.301988,-0.012573,-0.287234,0.196079,0.040103,0.059463,0.324868
3,0.310165,0.162723,-0.039514,0.008919,0.255538,-0.264964,0.289035,0.105345,-0.017594,0.053867,...,-0.111539,0.112330,-0.181848,-0.073778,-0.059616,0.045054,0.191947,0.286819,-0.067321,0.163990
4,0.167977,0.022978,0.033517,0.249408,0.063067,-0.408302,0.304360,0.120799,0.175538,-0.130975,...,-0.064510,0.084802,-0.092907,-0.162694,-0.093241,0.073792,-0.128327,0.185978,0.177096,-0.108767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,0.330300,0.295764,0.117434,-0.133158,0.259933,-0.036908,0.353735,-0.075970,-0.075929,-0.182939,...,-0.120197,-0.097423,-0.254647,-0.171841,-0.397316,-0.315863,0.028011,0.099914,0.159043,0.309869
12496,0.135886,0.188740,-0.247926,0.151798,-0.042878,-0.070221,0.372331,-0.003703,0.053845,-0.200384,...,-0.179420,0.138549,-0.259815,-0.162573,-0.340535,-0.471309,0.074000,0.156366,0.287940,0.295685
12497,0.156243,-0.004264,-0.043666,0.077168,-0.081637,0.376960,0.178357,0.198301,-0.329599,0.116169,...,0.012415,-0.176164,-0.050735,-0.229026,0.031625,-0.123867,0.405285,-0.052769,0.278198,0.177824
12498,0.095616,0.404711,-0.304204,0.147562,-0.119110,-0.189288,0.179682,-0.116130,0.041789,0.049917,...,-0.275982,0.024390,-0.267629,-0.287303,-0.189357,-0.462369,-0.034423,0.313417,0.192184,0.465953


In [9]:
x_train_searches, x_test_searches, y_train_searches, y_test_searches = train_test_split(feature_vectors_searches, df_tweets_searches['isBot'], test_size = 0.2)

# Hyperparameter Tuning
We shall perform hyperparameter tuning for the models that use BERT features as the features. The method we will use to perform this tuning is Bayesian Optimisation, which is more efficient in finding the best hyperparameters as compared to Random Search and Grid Search as those 2 methods treat each trial of hyperparameters independently while Bayesian optimization is an informed search in the hyperparameter space in order to optimize the objective function.

However, in the context of Random Forest, since most of the features are discrete, we used random search instead.

Then, the tuned models will be tested on an independent test set and the best performing model will be selected based on this test set.

(Note, the results are likely to be different each time the notebook is run)

## 07 Random Forest Classifier (BERT)
* Model (Random)
* Error Metrics

### Model (Random)

In [10]:
# Model
rf_search = RandomForestClassifier()

random_rf = {"n_estimators": [100, 200, 600, 800], 
             "max_features": ["auto", "sqrt", "log2"], 
             "max_depth": [10, 20, 40, 80], 
             "min_samples_split": [10, 40, 80, 100], 
             "bootstrap": [True, False]}

random_search = RandomizedSearchCV(estimator = rf_search,
                                   param_distributions = random_rf,
                                   scoring = 'f1',
                                   n_jobs = -1,
                                   cv = 5,
                                   n_iter = 3,
                                   random_state = 101,
                                   verbose = 2)

In [11]:
%%time

# Execute search
results_random_rf = random_search.fit(x_train_searches, np.ravel(y_train_searches))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
CPU times: total: 1min 51s
Wall time: 9min 48s


In [12]:
print('Best Score: %s' % results_random_rf.best_score_)
print('Best Hyperparameters: %s' % results_random_rf.best_params_)

Best Score: 0.7545117473905061
Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 80, 'bootstrap': False}


In [13]:
%%time

# y_prediction for the best model
y_pred_rf_searches = results_random_rf.predict(x_test_searches)

CPU times: total: 328 ms
Wall time: 334 ms


In [14]:
# Metrics for tuned random forest
accuracy = metrics.accuracy_score(y_test_searches, y_pred_rf_searches)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test_searches, y_pred_rf_searches)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test_searches, y_pred_rf_searches)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test_searches, y_pred_rf_searches)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test_searches, y_pred_rf_searches)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test_searches, y_pred_rf_searches)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test_searches, y_pred_rf_searches)
print(f'Classification Report: \n {report}')

Accuracy: 0.746
Log Loss: 8.77296722488801
ROC AUC: 0.7454309314038012
F1-score: 0.759013282732448
Precision: 0.7304601899196493
Recall: 0.7898894154818326
Classification Report: 
               precision    recall  f1-score   support

           0       0.76      0.70      0.73      1234
           1       0.73      0.79      0.76      1266

    accuracy                           0.75      2500
   macro avg       0.75      0.75      0.75      2500
weighted avg       0.75      0.75      0.75      2500



## 08 Adaboost Classifier (BERT)
* Model (Random)
* Error Metrics

### Model (Random)

In [15]:
adaboost_search = AdaBoostClassifier()

random_adaboost = {"n_estimators": [50, 100, 200, 500, 800], 
                   "learning_rate": [0.1, 1.0, 1.1, 1.2]}

random_search = RandomizedSearchCV(estimator = adaboost_search,
                                   param_distributions = random_adaboost,
                                   scoring = 'f1',
                                   n_jobs = -1,
                                   cv = 5,
                                   n_iter = 3,
                                   random_state = 101,
                                   verbose = 2)

In [16]:
%%time

results_random_adaboost = random_search.fit(x_train_searches, y_train_searches)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
CPU times: total: 12min 18s
Wall time: 29min 8s


In [17]:
print('Best Score: %s' % results_random_adaboost.best_score_)
print('Best Hyperparameters: %s' % results_random_adaboost.best_params_)

Best Score: 0.7271169467058299
Best Hyperparameters: {'n_estimators': 500, 'learning_rate': 0.1}


In [18]:
%%time

# y_prediction for the best model
y_pred_ada_searches = results_random_adaboost.predict(x_test_searches)

CPU times: total: 4.8 s
Wall time: 4.84 s


In [19]:
# Metrics
accuracy = metrics.accuracy_score(y_test_searches, y_pred_ada_searches)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test_searches, y_pred_ada_searches)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test_searches, y_pred_ada_searches)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test_searches, y_pred_ada_searches)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test_searches, y_pred_ada_searches)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test_searches, y_pred_ada_searches)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test_searches, y_pred_ada_searches)
print(f'Classification Report: \n {report}')

Accuracy: 0.7264
Log Loss: 9.449926602550315
ROC AUC: 0.7260991240804894
F1-score: 0.7350890782339271
Precision: 0.7211246200607903
Recall: 0.7496050552922591
Classification Report: 
               precision    recall  f1-score   support

           0       0.73      0.70      0.72      1234
           1       0.72      0.75      0.74      1266

    accuracy                           0.73      2500
   macro avg       0.73      0.73      0.73      2500
weighted avg       0.73      0.73      0.73      2500



## 9 XGBoost Classifier (BERT)
* Model (Random)
* Error Metrics

### Model (Random)

In [20]:
weight_train_searches = y_train_searches.value_counts()[0] / y_train_searches.value_counts()[1]

xgb_search = XGBClassifier(scale_pos_weight = weight_train_searches,
                           gamma = 0.1,
                           alpha = 0.5,
                           objective = "binary:logistic",
                           eval_metric = "logloss")

random_xgb = {"max_depth": [5, 10, 15, 20, 25], 
            "subsample": [0.6, 0.8, 1.0], 
            "eta": [0.01, 0.1, 0.5], 
            "colsample_bytree": [0.7, 0.8, 0.9, 1.0]}

random_search = RandomizedSearchCV(estimator = xgb_search,
                                   param_distributions = random_xgb,
                                   scoring = 'f1',
                                   n_jobs = -1,
                                   cv = 5,
                                   n_iter = 3,
                                   random_state = 101,
                                   verbose = 2)

In [21]:
%%time

results_random_xgb = random_search.fit(x_train_searches, y_train_searches)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
CPU times: total: 14min 4s
Wall time: 18min 2s


In [22]:
print('Best Score: %s' % results_random_xgb.best_score_)
print('Best Hyperparameters: %s' % results_random_xgb.best_params_)

Best Score: 0.7428479366373191
Best Hyperparameters: {'subsample': 0.8, 'max_depth': 25, 'eta': 0.01, 'colsample_bytree': 1.0}


In [23]:
%%time

# y_prediction for the best model
y_pred_xgb_searches = results_random_xgb.predict(x_test_searches)

CPU times: total: 469 ms
Wall time: 108 ms


In [24]:
# Metrics
accuracy = metrics.accuracy_score(y_test_searches, y_pred_xgb_searches)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test_searches, y_pred_xgb_searches)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test_searches, y_pred_xgb_searches)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test_searches, y_pred_xgb_searches)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test_searches, y_pred_xgb_searches)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test_searches, y_pred_xgb_searches)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test_searches, y_pred_xgb_searches)
print(f'Classification Report: \n {report}')

Accuracy: 0.74
Log Loss: 8.98020404116411
ROC AUC: 0.7393736189737327
F1-score: 0.7543461829176114
Precision: 0.7231884057971014
Recall: 0.7883096366508688
Classification Report: 
               precision    recall  f1-score   support

           0       0.76      0.69      0.72      1234
           1       0.72      0.79      0.75      1266

    accuracy                           0.74      2500
   macro avg       0.74      0.74      0.74      2500
weighted avg       0.74      0.74      0.74      2500

